<a href="https://colab.research.google.com/github/khuramgill/ML-DL/blob/main/Kaggle_Skill_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd

df = pd.read_csv('used_cars.csv')
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   brand         4009 non-null   object
 1   model         4009 non-null   object
 2   model_year    4009 non-null   int64 
 3   milage        4009 non-null   object
 4   fuel_type     3839 non-null   object
 5   engine        4009 non-null   object
 6   transmission  4009 non-null   object
 7   ext_col       4009 non-null   object
 8   int_col       4009 non-null   object
 9   accident      3896 non-null   object
 10  clean_title   3413 non-null   object
 11  price         4009 non-null   object
dtypes: int64(1), object(11)
memory usage: 376.0+ KB


In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Load the dataset
df = pd.read_csv('used_cars.csv')

# Convert columns to appropriate types
df['model_year'] = pd.to_numeric(df['model_year'], errors='coerce')
df['milage'] = pd.to_numeric(df['milage'].str.replace(',', ''), errors='coerce')
df['engine'] = pd.to_numeric(df['engine'].str.replace('L', ''), errors='coerce')
df['accident'] = pd.to_numeric(df['accident'], errors='coerce')
df['clean_title'] = pd.to_numeric(df['clean_title'], errors='coerce')
df['price'] = pd.to_numeric(df['price'].str.replace('$', '').str.replace(',', ''), errors='coerce')

# Assuming 'id' is present in the dataset to uniquely identify each car
if 'id' not in df.columns:
    df['id'] = range(len(df))  # If 'id' is not present, create a unique identifier for each row

# Split the data into features and target variable
X = df.drop(columns=['price'])
y = df['price']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the preprocessing for numeric and categorical features
numeric_features = ['model_year', 'milage', 'engine', 'accident', 'clean_title']  # Updated numeric features
categorical_features = ['brand', 'model', 'fuel_type', 'transmission', 'ext_col', 'int_col']  # Updated categorical features

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Create the model pipeline
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())])

# Train the model
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Combine the test IDs and predicted prices into a single DataFrame
submission = X_test[['id']].copy()
submission['price'] = y_pred

# Save the submission DataFrame to a CSV file
submission.to_csv('result.csv', index=False)

# Output evaluation metrics
print('Mean Squared Error:', mean_squared_error(y_test, y_pred))
print('R² Score:', r2_score(y_test, y_pred))


Mean Squared Error: 19499445903.112934
R² Score: 0.04599534608082112
